# Toronto Neighborhood Suggestions

In this Notebook, we are finding the best areas to live based on our preferences

1. Install required packages

In [1]:
import sys
!{sys.executable} -m pip install wikipedia
!{sys.executable} -m pip install lxml
!{sys.executable} -m pip install geopy
!{sys.executable} -m pip install folium

You are using pip version 18.0, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.0, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.0, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.0, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


2. Import required packages

In [2]:
import pandas as pd
import wikipedia as wp
import numpy as np
import requests
import folium
from pandas.io.json import json_normalize

3. Fetch the html from wikipedia page and assign to pandas dataframe

In [3]:
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
toronto_df = pd.read_html(html)[0]
print(toronto_df.shape)
toronto_df.head()

(289, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


4. Rename columns, cleanup **Not assigned**

In [4]:
toronto_df.rename(columns={"Postcode": "PostalCode", "Neighbourhood":"Neighborhood"}, inplace=True)
toronto_df = toronto_df[toronto_df.Borough != "Not assigned"]
toronto_df["Neighborhood"]= toronto_df["Neighborhood"].replace("Not assigned", toronto_df["Borough"])
toronto_df = toronto_df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
print (toronto_df.shape)

(103, 3)


5. Merge Geospatial data

In [5]:
geospatial_df = pd.read_csv('http://cocl.us/Geospatial_data')
geospatial_df.columns = ['PostalCode', 'Latitude', 'Longitude']
toronto_df = pd.merge(toronto_df, geospatial_df, on=['PostalCode'], how='inner')
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


6. Find geo coordinates of Toronto

In [6]:
from geopy.geocoders import Nominatim
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="toronto_neighborhood")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of the City of Toronto are 43.653963, -79.387207.


7. Visualize the neighborhoods in map

In [7]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

8. Setup Foursquare credentials

In [8]:
CLIENT_ID = 'CFGWCDIYPZJD2Q0VHIYY2RTBHOOMVTVSCJR0UADSMSUVPRLW' 
CLIENT_SECRET = 'XVW0VLNOTFGSBNB3RLOEWNVHXSVLRACYH0SGWGWAD2RH035Y' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CFGWCDIYPZJD2Q0VHIYY2RTBHOOMVTVSCJR0UADSMSUVPRLW
CLIENT_SECRET:XVW0VLNOTFGSBNB3RLOEWNVHXSVLRACYH0SGWGWAD2RH035Y


9. Get First Neighborhood

In [9]:
toronto_df.loc[0, 'Neighborhood']

'Rouge, Malvern'

10. Get geo coordinates of The Beaches

In [10]:
neighborhood_latitude = toronto_df.loc[0, 'Latitude'] 
neighborhood_longitude = toronto_df.loc[0, 'Longitude']
neighborhood_name = toronto_df.loc[0, 'Neighborhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge, Malvern are 43.806686299999996, -79.19435340000001.


11. Frame the Foursquare url

In [11]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=CFGWCDIYPZJD2Q0VHIYY2RTBHOOMVTVSCJR0UADSMSUVPRLW&client_secret=XVW0VLNOTFGSBNB3RLOEWNVHXSVLRACYH0SGWGWAD2RH035Y&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

12. Get the json response from Foursquare

In [12]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c9545841ed219272c816997'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

13. Get category type

In [13]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

14. Get Nearby venues

In [14]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) 

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056


In [15]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


15. Get Nearby venues

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )
print(toronto_venues.shape)

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The D

In [18]:
print(toronto_venues.shape)
toronto_venues.head()

(2231, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa


16. Group Neighborhood venues

In [29]:
toronto_venues.groupby('Neighborhood').count().sort_values('Venue', ascending=False).head(10)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
St. James Town,100,100,100,100,100,100
"Ryerson, Garden District",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Design Exchange, Toronto Dominion Centre",100,100,100,100,100,100
"Harbourfront East, Toronto Islands, Union Station",100,100,100,100,100,100
"Chinatown, Grange Park, Kensington Market",98,98,98,98,98,98
Stn A PO Boxes 25 The Esplanade,95,95,95,95,95,95


In [20]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 270 uniques categories.


In [21]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
toronto_onehot.shape

(2231, 270)

In [23]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').sum().reset_index()
toronto_grouped.replace(0,np.nan).dropna(axis=1,how="all").replace(np.nan, 0)

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,"Bathurst Manor, Downsview North, Wilson Heights",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,"Birch Cliff, Cliffside West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
toronto_grouped.shape

(99, 270)

In [40]:
toronto_grouped.iloc[1:,1:].sum().sort_values(ascending=False).head(11)

Score                 316
Coffee Shop           182
Café                   98
Restaurant             56
Pizza Place            56
Park                   52
Italian Restaurant     49
Bakery                 45
Sandwich Place         42
Hotel                  41
Bar                    38
dtype: int64

In [25]:
def get_suggestions(preferences, num_places):
    toronto_grouped['Score'] = 0
    columns = np.append(my_preferences['Category'].values, 'Score')
    columns = np.insert(columns, 0, 'Neighborhood')
    for index, preference in preferences.iterrows():
        toronto_grouped['Score'] += (toronto_grouped[preference['Category']] * preference['Weight'])
    return toronto_grouped[columns].sort_values('Score', ascending=False).reset_index(drop=True).head(num_places)

In [26]:
print('The categories are {}'.format(toronto_venues['Venue Category'].unique()))

The categories are ['Fast Food Restaurant' 'Bar' 'Pizza Place' 'Electronics Store' 'Spa'
 'Mexican Restaurant' 'Rental Car Location' 'Medical Center'
 'Breakfast Spot' 'Coffee Shop' 'Korean Restaurant' 'Hakka Restaurant'
 'Caribbean Restaurant' 'Athletics & Sports' 'Thai Restaurant' 'Bank'
 'Bakery' 'Fried Chicken Joint' 'Playground' 'Department Store'
 'Convenience Store' 'Chinese Restaurant' 'Hobby Shop' 'Discount Store'
 'Bus Station' 'Intersection' 'Bus Line' 'Metro Station' 'Park'
 'Soccer Field' 'Motel' 'Movie Theater' 'American Restaurant' 'Café'
 'General Entertainment' 'Skating Rink' 'College Stadium'
 'Indian Restaurant' 'Latin American Restaurant' 'Vietnamese Restaurant'
 'Pet Store' 'Thrift / Vintage Store' 'Furniture / Home Store'
 'Gaming Cafe' 'Sandwich Place' 'Middle Eastern Restaurant'
 'Shopping Mall' 'Auto Garage' 'Lounge' 'Italian Restaurant'
 'Noodle House' 'Pharmacy' 'Grocery Store' 'Golf Course' 'Pool'
 'Mediterranean Restaurant' 'Dog Run' 'Toy / Game Store' 'Bur

In [27]:
my_preferences = pd.DataFrame(columns = ['Category', 'Weight'])
my_preferences['Category'] = ['American Restaurant', 'Bakery', 'Breakfast Spot', 'Fast Food Restaurant', 'Coffee Shop']
my_preferences['Weight'] = [1, 1, 1, 1, 1]
suggestions_df = get_suggestions(my_preferences, 5)
suggestions_df

,Neighborhood,American Restaurant,Bakery,Breakfast Spot,Fast Food Restaurant,Coffee Shop,Score
0,"Commerce Court, Victoria Hotel",4,3,1,2,13,23
1,"Design Exchange, Toronto Dominion Centre",4,2,1,1,13,21
2,Stn A PO Boxes 25 The Esplanade,1,2,2,3,9,17
3,St. James Town,2,3,4,1,7,17
4,"Harbourfront East, Toronto Islands, Union Station",0,2,0,1,13,16


In [ ]:
suggestions_df = pd.merge(suggestions_df, toronto_df, on=['Neighborhood'], how='inner')
suggestions_df

In [ ]:
map_suggestions = folium.Map(location=[latitude, longitude], zoom_start=14)
for index, lat, lng, borough, neighborhood in zip(suggestions_df.index, suggestions_df['Latitude'], suggestions_df['Longitude'], suggestions_df['Borough'], suggestions_df['Neighborhood']):
    label = '{}, {}, {}'.format(index + 1, neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_suggestions)
map_suggestions